<a href="https://colab.research.google.com/github/maimonahST/Signal_It/blob/main/DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Define Functions

In [1]:
! pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.3 MB/s eta 0:00:00


In [22]:
! pip install PyDrive2

In [42]:
import numpy as np
from keras.models import load_model
import joblib
import mne
from google.colab import drive
from google.colab import files
import os

def Compute_ICA(file_path):
   # Set the verbosity level to "ERROR" to suppress most messages
    mne.set_log_level('ERROR')

    # Read the raw file
    raw = mne.io.read_raw_bdf(file_path, preload=True)

    # Apply ICA to remove artifacts
    ica = mne.preprocessing.ICA(n_components=32, random_state=42)
    raw.resample(250)
    raw.filter(5, 25)
    ica.fit(raw)

    # Extract features (you may need to customize this step based on your specific needs)
    features = ica.get_components()

    # flatten
    features = features.reshape(1, -1)

    # print(features.shape)

    return features


def Diagnose(test_sample):

  # File paths
  model_paths = [
      '/content/drive/MyDrive/GP/ICA/Adaboost_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/BernoulliNB_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/RF_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/XGBoost_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/gaussianNB_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/knn_ica.pkl',
      '/content/drive/MyDrive/GP/ICA/dt_ica.pkl'
  ]

  # Load pre-trained models
  models = []
  for path in model_paths:
      try:
          model = joblib.load(path)
          models.append(model)
      except Exception as e:
          print(f"Failed to load model from {path}: {e}")


  # Average the predictions
  avg_preds = np.mean([model.predict(test_sample) for model in models], axis=0)

  # Calculate final prediction and accuracy
  final_pred = (avg_preds > 0.5).astype(int)  # Assuming binary classification
  accuracy = np.mean(avg_preds)  # Average confidence for the positive class

  diagnosis = "Healthy" if final_pred == 0 else "Stroke"
  accuracy = round(accuracy * 100, 2)  # Convert to percentage and round

  # Adjust accuracy for "Healthy" diagnosis
  if diagnosis == "Healthy":
      accuracy = 100 - accuracy

  return diagnosis, accuracy


# # Function to select file from Google Drive
def select_file():
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User selected file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    return list(uploaded.keys())[0]


def select_file_from_drive_folder():

    folder_path = '/content/drive/MyDrive/GP/GP-Data/Ischaemia'

    # Mount Google Drive
    drive.mount('/content/drive')

    # List files in the specified folder
    folder_files = os.listdir(folder_path)
    for i, file in enumerate(folder_files):
        print(f'{i + 1}: {file}')

    # Select a file from the list
    index = int(input("Enter the file number you want to select: ")) - 1
    print("*************************************************")
    if 0 <= index < len(folder_files):
        selected_file_name = folder_files[index]
        final_path = os.path.join(folder_path, selected_file_name)
        return final_path
    else:
        print("Invalid selection.")
        return None


# Demo

In [44]:
# Select file (Stroke)
file_path = select_file_from_drive_folder()

# Preprocess raw input
test_sample = Compute_ICA(file_path)

# Diagnose the file
diagnosis, accuracy = Diagnose(test_sample)

print("************* Diagnose Result : ***************")
print(f"diagnosis is : {diagnosis}")
print(f"accuracy is : {accuracy}%")
print("***********************************************")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1: P1_TD1.bdf
2: P4a_TD1.bdf
3: P4b_TD1.bdf
4: P9_TD1.bdf
5: P12a_TD2.bdf
6: P12b_TD1.bdf
7: P15_TD1.bdf
8: P16_TD1.bdf
9: P19a_TD1.bdf
10: P18_TD1.bdf
11: P19b_TD1.bdf
12: P25a_TD1.bdf
13: P25b_TD1.bdf
14: P26_TD1.bdf
Enter the file number you want to select: 1
*************************************************
************* Diagnose Result : ***************
diagnosis is : Stroke
accuracy is : 85.71%
***********************************************
